In [1]:
using CSV, DataFrames, JuMP, Gurobi

[ Info: Precompiling DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]


In [14]:
pwd()

"C:\\Users\\Waz\\Documents\\Classes\\Opt\\OptProject"

In [15]:
classroomCapacity = CSV.read(".\\Data\\1000\\1000classroomCapacity.csv", DataFrame)[:,2:end];
daysAvailable = CSV.read(".\\Data\\1000\\1000daysAvailable.csv", DataFrame)[:,2:end];
studentClasses = CSV.read(".\\Data\\1000\\1000studentClasses.csv", DataFrame)[:,2:end]; # Q
subjectClassrooms = CSV.read(".\\Data\\1000\\1000subjectClassrooms.csv", DataFrame)[:,2:end]; # R
classPeriods = CSV.read(".\\Data\\1000\\1000classPeriods.csv", DataFrame)[:,2:end]; # T

In [20]:
classPeriods

Row,Period_0,Period_1,Period_2,Period_3,Period_4,Period_5,Period_6,Period_7
,String31?,String15?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?
1,0,1,1,0,0,0,0,0
2,1,0,1,1,0,0,1,0
3,0,0,1,0,0,1,0,1
4,0,1,0,1,0,0,0,0
5,1,1,0,1,0,0,0,1
6,1,0,0,1,1,0,0,0
7,0,0,0,1,0,0,1,0
8,0,1,1,0,0,0,1,1
9,0,1,0,0,0,1,0,1


In [4]:
model = Model(Gurobi.Optimizer);

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22


In [19]:
I = 1:size(classroomCapacity,1)
J = 1:8
K = 1:size(studentClasses,1)
D = 1:2
P = 1:15
Q = Matrix(studentClasses)# k arrays (matrix) each person takes
R = Matrix(subjectClassrooms)# p arrays of subjects and their classrooms (matrix)
T = Matrix(classPeriods)# matrix of which periods each subject can be in;

34×8 Matrix{Any}:
 "0"                  …  0         0         0
 "1"                     0         1         0
 "0"                     1         0         1
 "0"                     0         0         0
 "1"                     0         0         1
 "1"                  …  0         0         0
 "0"                     0         1         0
 "0"                     0         1         1
 "0"                     1         0         1
 "0"                     1         0         1
 "0"                  …  0         0         0
 "1"                     0         1         0
 "1"                     0         0         0
 ⋮                              ⋱  ⋮                   
 "Optimization"           missing   missing   missing
 "Machine Learning"       missing   missing   missing
 "Business"               missing   missing   missing
 "Astro Engineering"  …   missing   missing   missing
 "Aero Engineering"       missing   missing   missing
 "Political Science"      missing   missing  

### Variables

In [ ]:
#@variable(model, x[I, J], Bin); # classroom
@variable(model, s[I, J, K, P], Bin); # student
@variable(model, S[I, J, D, P], Bin); # subject

## Constraints

In [ ]:
@objective(model, Min, sum(S[i, j, d, p] for i in I for j in J for d in D for p in P))

#### Student constraints

In [ ]:
# student athlete cannot have class period 8
# student in chorus 
for k in K
    for i in I
        for j in J
            if studentClasses[k, 16] == 1 # if they are athlete
                if j == 8
                    @constraint(model, [p = P], s[i, j, k, p] = 0)
                end
            end
            if studentClasses[k, 17] == 1
                if j == 7 | j == 8
                    @constraint(model, [p = P], s[i, j, k, p] = 0)
            end
        end
    end
end

In [ ]:
# each student only one class at same time
@constraint(model, [j = J, k = K, p = P], sum(s[i, j, k, p] for i in I) == 1)

In [ ]:
# each student gets all of their classes
# Q is a matrix that is 1000x15 where there are 15 classes, binary if student needs it or not
@constraint(model, [i = I, j = J, k = K], sum(s[i, j, k, p]*Q[k, p] for p in P) == 5)

#### Classroom Capacities

In [ ]:
# this works because we have the next constraint
@constraint(model, [i = I, j = J, p = P], sum(s[i, j, k, p] for k in K) <= classroomCapacity[i, 1])

#### Period Constraints

In [ ]:
# T is a 15 x 8 matrix, p x j
# subjects only in certain time periods allowed
# for a given subject, classroom, day, sum time periods and make sure it is less than total classrooms for subject
@constraint(model, [p = P, i = I, d = D], sum(S[i, j, d, p]*T[p, j] for p in P) <= sum(T[p, j] for j in J))
for i in I
    for j in J
        for d in D
            for p in P
                if T[p, j] == 0 # if subject is not given during that time period
                    S[i, j, d, p] == 0 # make sure Subject is 0
                end
            end
        end
    end
end

#### Subject Constraints

In [ ]:
#i, j, d, p
# only one subject can be in one classroom during one time period on certain day
@constraint(model, [p = P, d = D, j = J], sum(S[i, j, d, p] for i in I) == 1)

# if subject is not given on a certain day, make sure it is 0
#@constraint(model, [i = I, p = P, d = D, j = J], S[i, j, d, p] = 0 if daysAvailable[p, 2] == 1)
@constraint(model, [i = I, p = P, d = D, j = J], S[i, j, d, p] = 0, when=daysAvailable[p, 2] == 1)

In [ ]:
# ensuring subjects are only in their classrooms
# R is a 15 
@constraint(model, [i = I, j = J, d = D], sum(S[i, j, d, p]*R[p, i] for p in P) <= sum(R[p,i] for i in I))
for i in I
    for j in J
        for d in D
            for p in P
                if R[p, i] == 0 # if subject is not in a classroom
                    S[i, j, d, p] == 0 # make sure subject is 0
                end
            end
        end
    end
end

In [ ]:
optimize!(model)